In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from gpt3forchem.data import get_bandgap_data
from gpt3forchem.api_wrappers import fine_tune, query_gpt3, extract_inverse_prediction
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
import time

In [4]:
data = get_bandgap_data()

In [7]:
train_data, test_data = train_test_split(data, train_size=500, test_size=100, random_state=42)

In [12]:
data = data.groupby('smiles').agg('mean').reset_index()

In [13]:
data

,smiles,mw,atoms,heavy_atoms,heteroatoms,rotatable_bonds,stereocenters,rings,hbond_acceptors,hbond_donors,significant_negative_wavenumbers,nonunique_smiles,GFN2_TOTAL_ENERGY,GFN2_ATOMIC_ENERGY,GFN2_FORMATION_ENERGY,GFN2_TOTAL_ENTHALPY,GFN2_TOTAL_FREE_ENERGY,GFN2_DIPOLE_X,GFN2_DIPOLE_Y,GFN2_DIPOLE_Z,GFN2_DIPOLE_TOT,GFN2_QUADRUPOLE_XX,GFN2_QUADRUPOLE_XY,GFN2_QUADRUPOLE_YY,GFN2_QUADRUPOLE_XZ,GFN2_QUADRUPOLE_yz,GFN2_QUADRUPOLE_ZZ,GFN2_ROT_CONSTANT_A,GFN2_ROT_CONSTANT_B,GFN2_ROT_CONSTANT_C,GFN2_ENTHALPY_VIB,GFN2_ENTHALPY_ROT,GFN2_ENTHALPY_TRANSL,GFN2_ENTHALPY_TOT,GFN2_HEAT_CAPACITY_VIB,GFN2_HEAT_CAPACITY_ROT,GFN2_HEAT_CAPACITY_TRANSL,GFN2_HEAT_CAPACITY_TOT,GFN2_ENTROPY_VIB,GFN2_ENTROPY_ROT,GFN2_ENTROPY_TRANSL,GFN2_ENTROPY_TOT,GFN2_HOMO_ENERGY,GFN2_LUMO_ENERGY,GFN2_HOMO_LUMO_GAP,GFN2_FERMI_LEVEL,GFN2_DISPERSION_COEFFICIENT_MOLECULAR,GFN2_POLARIZABILITY_MOLECULAR,DFT_TOTAL_ENERGY,DFT_ATOMIC_ENERGY,DFT_FORMATION_ENERGY,DFT_DIPOLE_X,DFT_DIPOLE_Y,DFT_DIPOLE_Z,DFT_DIPOLE_TOT,DFT_ROT_CONSTANT_A,DFT_ROT_CONSTANT_B,DFT_ROT_CONSTANT_C,DFT_XC_ENERGY,DFT_NUCLEAR_REPULSION_ENERGY,DFT_ONE_ELECTRON_ENERGY,DFT_TWO_ELECTRON_ENERGY,DFT_HOMO_ENERGY,DFT_LUMO_ENERGY,DFT_HOMO_LUMO_GAP
0,ClC(Cl)(Cl)Cl,151.875411,5.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-20.321877,-19.723397,-0.598480,-20.306790,-20.342824,0.001000,-0.001000,0.000000,0.001000,0.000000,-0.002333,0.000333,-0.000333,0.000000,-0.000333,0.057007,0.056986,0.056966,2020.639333,888.752,1481.254,4390.645333,13.116667,2.981,4.968,21.065667,11.236667,23.619667,40.983,75.839333,-0.463514,-0.275625,0.187889,-0.369569,1971.705218,67.656487,-1878.242035,-1877.752698,-0.489338,-0.000788,0.000230,0.000397,0.002485,0.057796,0.057774,0.057755,-86.472489,439.716519,-3468.214852,1236.729520,-0.398327,0.038182,0.436509
1,Clc1c(Cl)c(Cl)c2c(c1Cl)Oc1c(Cl)c(Cl)c(Cl)c(Cl)...,455.740651,22.0,22.0,10.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,-70.262982,-64.914971,-5.348011,-70.155416,-70.227767,-0.001000,0.000000,-0.000667,0.013000,-7.717333,-0.080333,-3.637333,0.912333,-1.719333,11.354667,0.008804,0.003158,0.002324,11472.794667,888.752,1481.254,13842.800667,67.298667,2.981,4.968,75.247667,72.482000,35.548667,44.245,152.276000,-0.408186,-0.305303,0.102883,-0.356744,22553.834251,235.252211,-4287.448511,-4283.439749,-4.008762,-0.001120,0.000205,-0.000142,0.002009,0.008901,0.003193,0.002352,-222.707722,3029.293594,-11943.469811,4849.451783,-0.310438,-0.004088,0.306350
2,FC(F)(F)c1nnc(/N=c2\ssnc2Cl)s1,303.892571,16.0,16.0,11.0,1.0,0.0,2.0,7.0,0.0,0.0,0.0,-50.896841,-47.169694,-3.727147,-50.820243,-50.878942,2.554333,-0.517667,0.233667,2.618333,-12.320667,9.327000,1.774333,-2.230333,-0.078333,10.546333,0.029477,0.005441,0.004706,7371.973000,888.752,1481.254,9741.979333,45.289000,2.981,4.968,53.238333,46.498000,34.025333,43.020,123.543000,-0.419630,-0.352957,0.066673,-0.386294,9791.448287,155.507219,-2362.480791,-2359.828013,-2.652778,0.799891,-0.305843,0.064215,0.864192,0.028218,0.005358,0.004680,-137.440788,1595.941281,-6432.895594,2611.921938,-0.321656,-0.059174,0.262482
3,O=C(C(=O)c1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl)c1c(F)c(F)...,469.826109,26.0,26.0,12.0,3.0,0.0,2.0,2.0,0.0,0.0,0.0,-84.458157,-78.150689,-6.307469,-84.327528,-84.408154,-1.925667,0.022000,-0.100000,2.224333,2.082333,0.328333,0.587333,1.856667,-4.269667,-2.670333,0.009149,0.003053,0.002561,13916.653667,888.752,1481.254,16286.660333,80.516000,2.981,4.968,88.465000,88.959000,36.408000,44.326,169.692667,-0.431704,-0.354521,0.077182,-0.393113,21181.533384,223.979499,-3482.061053,-3477.142356,-4.918697,-1.116850,0.384102,0.082660,1.698089,0.009198,0.002986,0.002516,-205.223668,3300.275075,-11375.138430,4798.047692,-0.344516,-0.042980,0.301536
4,O=C1C(=O)C(Cl)=C(Cl)C(Cl)=C1Cl,243.865240,12.0,12.0,6.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,-39.115145,-36.225093,-2.890052,-39.058615,-39.109900,-0.761000,3.676500,1.367500,3.995000,3.006000,3.314000,-6.986000,-1.832500,-5.471500,3.980000,0.023808,0.016869,0.009873,5579.678500,888.752,1481.254,7949.684500,33.530000,2.981,4.968,41.479000,34.560500,

In [14]:
PROMPT_TEMPLATE_bandgap_inverse = "What is a molecule with a bandgap of {} eV###"
COMPLETION_TEMPLATE_bandgap_inverse= "{}@@@"


def generate_inverse_photoswitch_prompts(data: pd.DataFrame, representation: str = 'smiles') -> pd.DataFrame:
    prompts = []
    completions = []
    smiles = []
    for i, row in data.iterrows():
    
        prompt = PROMPT_TEMPLATE_bandgap_inverse.format(
               np.round(row['GFN2_HOMO_LUMO_GAP']* 27.2114, 1)
            )

        completion = COMPLETION_TEMPLATE_bandgap_inverse.format(row[representation])
        prompts.append(prompt)
        completions.append(completion)
        smiles.append(row["smiles"])

    prompts = pd.DataFrame({"prompt": prompts, "completion": completions, "SMILES": smiles})

    return prompts


In [15]:
train_prompts, test_prompts = generate_inverse_photoswitch_prompts(train_data), generate_inverse_photoswitch_prompts(test_data)

In [16]:
train_prompts

,prompt,completion,SMILES
0,What is a molecule with a bandgap of 2.1 eV###,[H]c1c([H])c(-n2nc(C(=O)N([H])C([H])([H])C([H]...,[H]c1c([H])c(-n2nc(C(=O)N([H])C([H])([H])C([H]...
1,What is a molecule with a bandgap of 1.9 eV###,[H]c1c([H])c([H])c2c(c1[H])C(=O)C1=[N+]([O-])[...,[H]c1c([H])c([H])c2c(c1[H])C(=O)C1=[N+]([O-])[...
2,What is a molecule with a bandgap of 1.9 eV###,[H]c1nc2c(N3C([H])([H])C([H])([H])C([H])(C#N)C...,[H]c1nc2c(N3C([H])([H])C([H])([H])C([H])(C#N)C...
3,What is a molecule with a bandgap of 3.0 eV###,[H]c1c([H])c(S(=O)(=O)N([H])C([H])([H])C([H])(...,[H]c1c([H])c(S(=O)(=O)N([H])C([H])([H])C([H])(...
4,What is a molecule with a bandgap of 3.5 eV###,[H]OC(=O)C([H])([H])c1c([H])c([H])c(OC([H])([H...,[H]OC(=O)C([H])([H])c1c([H])c([H])c(OC([H])([H...
...,...,...,...
495,What is a molecule with a bandgap of 2.0 eV###,[H]c1nc(-c2c([H])c(F)c(F)c([H])c2F)c(-c2c([H])...,[H]c1nc(-c2c([H])c(F)c(F)c([H])c2F)c(-c2c([H])...
496,What is a molecule with a bandgap of 2.7 eV###,[H]c1c([H])c(C(=O)n2c(C([H])([H])C([H])([H])C(...,[H]c1c([H])c(C(=O)n2c(C([H])([H])C([H])([H])C(...
497,What is a molecule with a bandgap of 1.8 eV###,[H]c1c([H])c([H])c(-c2nc(N([H])c3c([H])c([H])c...,[H]c1c([H])c([H])c(-c2nc(N([H])c3c([H])c([H])c...
498,What is a molecule with a bandgap of 2.0 eV###,[H]c1c([H])c(OC([H])([H])[H])c(C([H])(C([H])([...,[H]c1c([H])c(OC([H])([H])[H])c(C([H])(C([H])([...


In [17]:
def train_inverse_model(train_prompts, representation):

    train_size = len(train_prompts)

    filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
    train_filename = f"run_files/{filename_base}_train_prompts_bandgap_inverse_{representation}_{train_size}.jsonl"

    train_prompts.to_json(train_filename, orient="records", lines=True)

    modelname = fine_tune(train_filename, train_filename, "ada")
    return modelname, train_filename

In [20]:
modelname, train_filename = train_inverse_model(train_prompts, 'smiles')

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

In [23]:
train_prompts['prompt'].iloc[0]

'What is a molecule with a bandgap of 2.1 eV###'

In [27]:
from gpt3forchem.output import test_inverse_bandgap

In [64]:
res = test_inverse_bandgap(test_prompts.iloc[:10], modelname, train_prompts['SMILES'].iloc[:10], 0.75, max_tokens=250)

2022-11-25 11:47:58.185 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c(C([H])([H])C([H])([H])C([H])([H])C([H])([H])N(c2nc3c([H])c([H])c([H])c([H])c3n2C([H])([H])[H])S(=O)(=O)C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H
2022-11-25 11:47:58.186 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c(C([H])([H])C([H])([H])C([H])([H])C([H])([H])N(c2nc3c([H])c([H])c([H])c([H])c3n2C([H])([H])[H])S(=O)(=O)C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H
2022-11-25 11:47:58.190 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frec

In [65]:
res

{'meta': {'temperature': 0.75,
  'max_tokens': 250,
  'model': 'ada:ft-lsmoepfl-2022-11-24-17-26-02'},
 'predictions': array(['[H]c1c([H])c(C([H])([H])C([H])([H])C([H])([H])C([H])([H])N(c2nc3c([H])c([H])c([H])c([H])c3n2C([H])([H])[H])S(=O)(=O)C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])N([H])[H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H',
        '[H]c1nc(N([H])c2nc([H])c(N3C([H])([H])C([H])([H])C([H])([H])N(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C3([H])[H])C([H])([H])C([H])([H])C3([H])[H])c([H])c2[H])c(Cl)c([H])c1Cl',
        '[H]c1sc(C([H])([H])n2c([H])nc([H])c2C([H])([H])[H])nc1C(=O)c1c([H])c([H])c(C([H])([H])[H])c([H])c1[H]',
        '[H]c1c([H])c(C([H])(C([H])([H])[H])C([H])([H])[H])c(C(=O)N([H])C(=O)N([H])c2c([H])c([H])c([H])c([H])c2[H])c1N([H])C([H])([H])C2=O',
        '[H]c1c([H])c([H])c(C(=O)OC([H])([H])C([H])([H])C([H])([H])N([H])C(=O)c2c(C([H])

In [44]:
from rdkit import Chem

In [47]:
mol = Chem.MolFromSmiles('[H]c1c([H])c([H])c(C([H])([H])N([H])C(=O)N([H])c2c([H])c([H])c(C(=O)N([H])c3c([H])c([H])c(OC([H])([H])[H]')

In [60]:
from gpt3forchem.output import FrechetBenchmark

In [62]:
frechet_benchmark = FrechetBenchmark(train_prompts['SMILES'].iloc[:10], sample_size=sum(res['valid_smiles']))

In [63]:
frechet_benchmark.score(res['predictions'][res['valid_smiles']])

2022-11-25 11:46:44.094 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/fcd/FCD.py:194: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  gen_mol_act = model.predict_generator(


(84.54795301593674, 4.531668258862096e-08)